In [ ]:
knitr::opts_chunk$set(echo = FALSE)
suppressMessages(expr =  {
  if ("patchwork" %in% row.names(installed.packages()) == FALSE) {
    install.packages("patchwork")
  }
  if ("bayesplay" %in% row.names(installed.packages()) == FALSE) {
    install.packages("bayesplay")
  }
  if ("polspline" %in% row.names(installed.packages()) == FALSE) {
    install.packages("polspline")
  }

  if ("IRdisplay" %in% row.names(installed.packages()) == TRUE) {
    display_markdown <<- \(x) IRdisplay::display_markdown(as.character(x))
    display_html <<- \(x) IRdisplay::display_html(as.character(x))
  } else {
    display_markdown <<- knitr::asis_output
    display_html <<- knitr::asis_output
  }

  library(tidyverse)
  library(polspline)
  library(patchwork)
  library(magrittr)
  library(bayesplay)
})

table_format <- "html"


# Null-hypothesis significance testing

Before diving into Bayesian hypothesis testing, it's worth spending a little time going over Frequentist null-hypothesis significance testing. The reason for this is that I want it to be clear that Bayesian methods and Frequentist methods aren't just two different ways of answering the same question. Rather, I want it to be clear that Bayesian methods and Frequentist methods are asking **fundamentally different questions**. For this reason, we'll start right back at the beginning with *p* values.

The American Statistical Association (ASA) defines a *p* value as:

> the probability under a specified statistical model that a statistical summary of the data (e.g., the sample mean difference between two compared groups) would be equal to or more extreme than its observed value ([Wasserstein and Lazar 2016](https://doi.org/10.1080/00031305.2016.1154108))

While this is a perfectly acceptable definition, it is maybe a little tricky to understand. The main reason for this is that the definition contains at least one *ill-defined concept* ("probability") and one tricky concept ("specified statistical model"). To understand what a *p* value really is, we're going to have to unpack both of these ideas. Along the way, we're going to learn about some other concepts that will also help us understand *Frequentist* inference. And a good grounding in Frequentist inference will also help us understand the distinction between Frequentist inference and Bayesian inference.

## Probability

Most people think of *probability* as a mathematical concept. In a sense it is, but it is also a deeply *philosophical* concept. We deploy the word *probability* in many different *kinds* of situations, and it's not clear whether we mean the same thing in each of them. Some examples of where we use the word probability are when we ask questions like: What is the probability of getting heads on repeated tosses of a fair coin? What is the probability that it will rain tomorrow? What is the probability the accused committed the crime? The word *probability* seemingly refers to different things in each of these situations.

For example, we might suggest that the probability of the getting heads is 0.5, where this 0.5 refers to the *long-run relative frequency* of getting heads. That is, if we were to toss a coin many many times then on around 0.5 (i.e., half) of the tosses the coin would come up heads.

We might use a different notion when thinking about the case of somebody accused of a crime. We might say something like, "we are 90% sure" (probability of .9) that the criminal committed the crime. But what does "90% sure" mean. Does it make sense to think of it as the *relative frequency*? If not, then how else might we think of it? We might, for example, think of it as a *credence* or a *degree of belief* that the proposition is true. Or we might think of it as a *degree of support*. That is, we might say that the available evidence supports the hypothesis that the accused committed the crime with odds of 9-to-1.

This list isn't meant to be exhaustive. The aim is just to highlight that we might sometimes mean different things when we think about probability. It pays to keep this in mind as we move through the course.

## Probability and *p* values

Now that we know that *probability* can mean different things in different situations, what notion of *probability* is at play in ASA's definition of the *p* value? The common view is to say that it refers to *relative frequencies*. But relative frequencies of **what** over repeats of **what**?

We could possibly re-phrase that definition to say something like this:

> the p value refers to the relative frequency of obtaining a statistical summary of the data as large or larger than the observed value over hypothetical repeats of an experiment described by a specified statistical model

### Understanding the *p* through simulation

One method that I think is useful for understanding statistical concept is *simulation*. This is particularly true in the case of *p*-values, because I definition above refers to *hypothetical repeats*. Simulation means that we can just simulate those *repeats*. To understand how *p* values work, let's start with a little scenario:

> You've been given a device that can be used to find buried treasure. The device has a numbered dial on it, and there is a little arrow that can point at these numbers. The indicator never stays still, but swings around a bit. You don't know how the device works, except that it behaves *differently* around treasure compared with when there is no treasure present. How can you use this device to find treasure?

This seems like a hard problem. You know very little about the device. You don't know what it's meant to do when it finds treasure, and you don't know what it's meant to do when there isn't any treasure. So how do you go about using it to find treasure?

#### Finding treasure

The first step in using the device is to get a good description of what it does when there isn't any treasure around. To do this, you might just take your device somewhere without treasure. You can then just sit and watch the dial. After a long time watching it, you might notice that although the pointer swings around a lot, *on average* it points at zero. This one bit of information is enough to develop a treasure hunting strategy using this device.

The first step in the strategy is deciding how many readings to take on each hunt. Because the pointer swings around a lot, we'll need to take a couple of readings and then use these to work out an average (which we'll call $\bar{x}$). We're in a hurry so we'll take **10** readings on each hunt.

Next, we'll need to scale our average. If our average is 1, then is this close to 0? How about 0.5? Or 5? Or 15? It's impossible to know because you don't know range of the average dial's swings. So your scaling factor should be proportional to the magnitude of the average deviations you've observed (we'll call this scaling factor $s_{\bar{x}}$).

With this information in hand, we have enough to build a statistical model of our device's behaviour. To do this, we just go where there is no treasure and perform the following steps: 1) Take 10 readings; 2) work out an average ($\bar{x}$); 3) scale it by our scaling factor ($s_{\bar{x}}$); write down our scaled measurement (which we'll call $t$), and repeat! Once we've done this many many times, then we'll have a nice distribution or statistical model of how our device behaves when there isn't any treasure. Of course, we don't have to do this for real. We can just simulate it! Feel free to play around with the simulation, to change the numbers, and to see how this influences our statistical model.

In [ ]:
# run this chunk is to set up the simulation function

set.seed(612) # Set the seed for reproducibility

run_exp <- function(sample_size, average) {
  # define function for running and experiment

  # we don't know how much the pointer actually swings around,
  # so lets just pick a random range between 1 and 10!
  dev <- runif(1, 1, 10)
  min_possible <- average - dev
  max_possible <- average + dev

  # generate a sample of readings
  this_sample <- runif(sample_size, min_possible, max_possible)

  # make descriptive stats
  tibble::tibble(
    sample_mean = mean(this_sample), # our x-bar
    sample_sd = sd(this_sample), # and average deviation from x-bar
    n = sample_size, # the sample size
    se = sample_sd / sqrt(sample_size), # our scaling  factor
    t = sample_mean / se # our scaled value
  )
}




In [ ]:
# run this chunk to actually run the simulations!

# Set the number of experiments to simulate
# This can be set to a higher number if you want to run the simulation
# for longer
no_of_exps <- 1000

all_exps <- purrr::map_df(1:no_of_exps, function(x) {
  run_exp(sample_size = 10, average = 0) %>% dplyr::mutate(i = x)
})

# view the first 10 rows of the table
all_exps %>%
  dplyr::slice(1:10) %>%
  knitr::kable(
    digits = 2,
    format = table_format,
    col.names = c("Sample Mean", "Sample sd", "N", "Std Error", "t", "index"),
    caption = "First 10 rows of simulated experiment data",
    align = "c",
  ) %>%
  display_html()




In [ ]:

# make a histogram of the unscaled averages
mean_hist <- all_exps %>% ggplot(aes(x = sample_mean)) +
  geom_histogram(fill = "seagreen", bins = 100, na.rm = TRUE) +
  labs(x = "sample mean", y = "number of experiments") +
  xlim(c(
    -max(abs(all_exps$sample_mean)) * 1.10,
    max(abs(all_exps$sample_mean)) * 1.10
  )) +
  theme_minimal(14)

# make a histograme of the scaled averages
t_hist <- all_exps %>% ggplot(aes(x = t)) +
  geom_histogram(fill = "seagreen", bins = 100, na.rm = TRUE) +
  labs(x = "t stat", y = "number of experiments") +
  xlim(c(-4, 4)) +
  theme_minimal(14)

# convert the histogram of the scaled averages to a probability density
density_plot <- tibble(x = seq(-4, 4, length.out = 10000)) %>%
  mutate(y = polspline::dlogspline(x, polspline::logspline(all_exps$t))) %>%
  ggplot(aes(x = x, y = y)) +
  geom_line(colour = "darkblue") +
  geom_function(
    fun = dt, args = list(df = unique(all_exps$n) - 1),
    linetype = 2
  ) +
  labs(x = "t", y = "density") +
  xlim(c(-4, 4)) +
  theme_minimal(14)

(((mean_hist | t_hist) / density_plot) + plot_annotation(tag_levels = "A"))




In panel **A** we can see the distribution of the raw averges from our device. In panel **B** the averages have been scaled. Finally, in panel **C**, the histogram has been turned into a density plot. The blue line shows our scaled averages, and the dashed black line shows a *t* distribution. As we increase the number of experiments we simulate then the two lines should begin to overlap.

#### Using our device

We can use our statistical model of our device (our distribution of *t* values) to come up with a method for finding treasure. Our statistical model tells us what readings we'll see when **we haven't found treasure** and **how often** we'll see those readings. In the absence of treasure we'll see readings near the middle of the distribution very often and readings near the tails of the distribution less often. We might even say that, in the absence of treasure, it would be pretty **surprising** to see an extreme reading. Now we don't know anything about how the device behaves when it's around treasure, but we know what readings would be *surprising* if it **wasn't** around treasure.

We can use this fact to come up with a treasure hunting rule. When you see a **surprising** reading (that is, if our average of multiple readings, from a single hunt falls in the extreme tails) the we dig for treasure. When you see an **unsurprising** reading, move on to the next spot. Let's try it out!

In [ ]:
# trying out our treasure hunting device
set.seed(151)
n <- 10
generated_data <- rnorm(n, runif(1, -10, 10), runif(1))
x_bar <- mean(generated_data) # work out an average
s_x_bar <- sd(generated_data) / sqrt(length(generated_data))
t_value <- x_bar / s_x_bar # work out the scaled measurement

# now report it nicely

generated_data_txt <- paste0(round(generated_data, 2), collapse = "; ")
x_bar_txt <- "$\\bar{x}$"
s_x_bar_txt <- "$s_{\\bar{x}}$"

glue::glue("Our {n} measurements are:
{generated_data_txt}

Our {x_bar_txt} = {round(x_bar,3)}

Our {s_x_bar_txt} = {round(s_x_bar,3)}

This means that our scaled measurement, $t$ = {round(t_value,3)}") %>%
  display_markdown()




Once we have our scaled reading $t$, we can ask how **surprising** it is. To do this, we just compare it against the distribution of measurements that we generated when we weren't around treasure.

In [ ]:
# how surprising is our measurement

# first convert it to an absolute value
t_value <- abs(t_value)

larger_than_positive <- mean(all_exps$t > t_value)
smaller_than_negative <- mean(all_exps$t < -t_value)

further_from_zero <- larger_than_positive + smaller_than_negative
closer_to_zero <- 1 - further_from_zero

glue::glue(
  "{round(closer_to_zero * 100,2)}% of values from our simulation",
  " where closer to zero than our current value.
Only {round(further_from_zero * 100, 2)}% of values where further from ",
  "zero than our current value."
) %>%
  display_markdown()




Once we have a measurement of how surprising our value is, then we just need to set a threshold for when it's surprising enough to warrant digging. We'll call this threshold $\alpha$, and we'll set it to 5% (for literally no reason in particular).

Now let's try using the rule. We'll do another simulation. We'll simulate many many hunts, and on each hunt there either will be treasure or there won't be treasure. Treasure will occur with the probability of $Pr_{\mathrm{treasure}}$. We won't know this value, because we'll just randomly set it. For each hunt, we'll note down whether the rule told us to dig or move one. And we'll also record the ground truth to test the accuracy.

In [ ]:
# testing our rule
set.seed(14) # set seed for reproducibly
n_tests <- 1000 # set the number of tests

pr_treasure <- runif(1) # Set the probablity of finding treasure

# a function for simulating a treasure hunt
simulate_hunt <- function(pr_treasure) {

  # Decide whether this hunt has_treasure
  # This is the ground truth
  has_treasure <- ifelse(runif(1) < pr_treasure, 1, 0)

  generated_data <- rnorm(10, has_treasure, 1.5) # generate 10 readings

  # work out the scaled measurement and how suprising it is
  # and decide whether to dig or not!
  # For this we'll use an actual t-distribution, rather than the distribution
  # that we simulated earlier

  t.test(generated_data) %>%
    broom::tidy() %>%
    dplyr::mutate(
      has_treasure = ifelse(has_treasure == 1, "Yes", "No"),
      how.suprising = p.value,
      dig = ifelse(p.value < 0.05, "Yes", "No")
    ) %>%
    dplyr::select(has_treasure, dig)
}

set.seed(11) # set seed for reproducibly
test_hunts <- map_df(1:n_tests, function(x) simulate_hunt(pr_treasure))




In [ ]:
# look at the results of tests

test_hunts %>%
  slice(1:20) %>%
  dplyr::slice(1:10) %>%
  knitr::kable(
    format = table_format,
    col.names = c("Had treasure?", "Did dig?"),
    caption = "First 10 rows of simulated treasure hunt data",
    align = "c"
  ) %>%
  display_html()




To asses the usefulness of our rule, we can evaluate the accuracy of our rule. There are a few ways to do this. We can look at overall accuracy. We can look at how often we missed treasure when there was treasure. We can look how often we dug for treasure when there wasn't any. Let's take a look at some metrics.

In [ ]:

# create a function to make the metrics look nice

get_metrics <- function(test_hunts) {
  hunt_summary <- test_hunts %>%
    group_by(has_treasure, dig) %>%
    summarise(n = n(), .groups = "drop") %>%
    mutate(n = 100 * (n / sum(n)))

  accuracy <- test_hunts %>%
    mutate(acc = has_treasure == dig) %>%
    pull(acc) %>%
    mean()
  hunt_summary %>%
    knitr::kable(
      digits = 2,
      align = "c",
      format = table_format,
      col.names = c("Had treasure?", "Did dig?", "%"),
      caption = paste0(
        "Our treasure hunting metrics. Accuracy: ",
        round(accuracy, 2)
      )
    )
}




In [ ]:
# Get some metrics for your simulated hunt
test_hunts %>%
  get_metrics() %>%
  display_html()




The rule seems to work pretty well in terms of accuracy. But how much is accuracy dependent on the actual probability of finding treasure? Let's run two more quick simulations where we set the probability of treasure actually being present to 1 (treasure all the time) or 0 (treasure none of the time).

In [ ]:
test_hunts_nothing <- map_df(1:n_tests, function(x) simulate_hunt(0))
test_hunts_nothing %>%
  get_metrics() %>%
  display_html()




In [ ]:
test_hunts_everywhere <- map_df(1:n_tests, function(x) simulate_hunt(1))
test_hunts_everywhere %>%
  get_metrics() %>%
  display_html()




But maybe just looking at accuracy isn't the best. After all, there are two ways in which we can be wrong. We can dig when we're not meant to, and we can move on when there's actually treasure. So let's split that accuracy percentage (or rather the \[1 - accuracy\] or "error" percentage) into two: 1) Digging when there's no treasure, and 2) moving on without digging when there was treasure. Now let's adjust $Pr_{\mathrm{treasure}}$ and see how the two error rates fare.

In [ ]:

# function for working out the metrics
get_metrics2 <- function(data) {
  data %>%
    mutate(
      FA = has_treasure == "No" & dig == "Yes",
      MS = has_treasure == "Yes" & dig == "No"
    ) %>%
    summarise(FA = mean(FA) * 100, MS = mean(MS) * 100) %>%
    pivot_longer(1:2) %>%
    pull(value, name)
}

# work out the metrics
nowhere_mtrx <- test_hunts_nothing %>%
  get_metrics2()
everywhere_mtrx <- test_hunts_everywhere %>%
  get_metrics2()

# now print the results nicely

glue::glue("When there was no treasure at all then the **false alarm
rate** was {nowhere_mtrx[['FA']]} and the **miss rate** was
{nowhere_mtrx[['MS']]}

When there was treasure everywhere then the **false alarm rate** was
{everywhere_mtrx[['FA']]} and the **miss rate** was
{everywhere_mtrx[['MS']]}") %>%
  display_markdown()



We can see that no matter what we do, the false alarm rate (digging when there is no treasure) never goes above \~5%, which is the same value we set for $\alpha$. This is great because it means that we can with certainty set the upper bound of this error rate. And, we can do so knowing nothing about how much treasure there is to be found or how our device works in the presence of treasure. All we need is: 1) to know that *on average* the device points at zero when there's no treasure around and 2) to sit and watch the device for a long time and just record some scaled measurements that the device produces. In fact, we don't even need to do (2). We can just *pretend* to this by simulating the results, and we only need to input **one parameter**---the same value we that we needed for step 1. Everything else can just be made up.

I'm not going to talk much about the other error rate, because this isn't a course of frequentist inference. But we can estimate it based on some assumptions about how the device behaves *in the presence of treasure*. For example, if we know that treasure of a certain value results in the device pointing on average at 1, then we can calculate the **upper bound** of missing treasures smaller than that value. Trying to estimate the **upper bound** on this error rate is what you’re doing when you’re doing a **power analysis**. It’s generally **a lot** harder to estimate this, because it involves a lot of guesswork. In comparison, estimating the first error rate is rather trivial.

## Summary

What this rather long-winded demonstration was meant to show is that *p* values are very good at doing one thing. That thing is, controlling how often, in the long run, we’ll make a particular kind of error. Deployed in the right context, they’re very good at this. This all comes from a simple process: Setting the value of **one parameter**, running pretend experiments, and then comparing our data at hand to results obtained from our pretend experiments to **judge whether our data is surprising** or not.

Of course, our treasure-hunting scenario may not be exactly analogous to how science works. These means that deciding whether *p* values are useful or not is going to depend on how closely their real-world use case matches their ideal operating environment.

### A short note on confidence intervals

I'll mention confidence intervals only briefly, but they follow the *exact* same logic as *p*-values. Let’s say I collect some measurements, work out the average. I could scale this value with my scaling factor, could get a *t* value. I could then turn to my list of results from the pretend experiments (the sampling distribution) to work out my *p* value. However, I can also the sampling distribution to construct (the very poorly named) confidence interval.

How could we do this? Looking at the sampling distribution we constructed earlier we would see that values that are more than about 2.23 *t* units from 0 would be surprising. Using this information, I can ask myself the following question: If my device on average pointed at the current sample average, rather than zero, what data values would be surprising? The answer to this is, of course, values that are more than 2.23 *t* units from the sample mean. Having an answer in *t* units isn't very useful. But I know that I converted measurements to *t* units by scaling readings using the scaling factor $s_{\bar{x}}$. This means we can just un-scale the value in *t* units back into raw units using the scaling factor calculated from my sample. This means I can say that any values $\pm 2.23\cdot{}s_{\bar{x}}$ from the sample mean would be surprising. Any values less than this, or in this range, would be unsurprising if my device, on average, pointed at my current sample mean. I could draw a line through these values, put little tails on this line, and *hey presto* I have a confidence interval.

Now just to be clear, just like with a *p* value, a confidence interval tells you what values would be surprising/unsurprising on an assumption of a certain value of the parameter. For the *p* value you set the parameter to 0 (or wherever else the device points when no treasure is around). For the confidence interval, I just set the parameter value to the estimate obtained from the current sample, but it’s exactly the same idea.

Hopefully, this should make it clear what the confidence interval does and doesn't tell you. It doesn't tell you about the *probability of a parameter falling within a range* (the common misinterpretation). It tells you *frequency with which data from pretend experiments will fall within a particular range on the assumption that the parameter is equal to the observed value*. At no point are we making inferences about **parameters** or **true values** of parameters. We are holding parameters constant, doing pretend experiments, and then marking out the range of surprising and unsurprising data. 

Now that we're all on the same page about *p* values and confidence intervals, and we have a good idea of where they come from let us the a look at some criticisms of *p* values.